In [5]:
import torch
from transformer_lens import HookedTransformer
from transformer_lens import utils

from functools import partial

torch.set_grad_enabled(False)



In [6]:
model: HookedTransformer = HookedTransformer.from_pretrained('gpt2-small', device='cpu')


Loaded pretrained model gpt2-small into HookedTransformer


In [7]:
layer = 1
# prompt_pos = "Yes, I talk about wedding constantly"
# prompt_neg = "I do not talk about wedding constantly"
prompt_pos = "Love "
prompt_neg = "Hate"

In [8]:
logits, pos_cache = model.run_with_cache(prompt_pos)
h_p = pos_cache["resid_post", layer]

logits, neg_cache = model.run_with_cache(prompt_neg)
h_n = neg_cache["resid_post", layer]

print(h_p.shape, h_n.shape)
steering = h_p - h_n
steering.shape

torch.Size([1, 3, 768]) torch.Size([1, 3, 768])


torch.Size([1, 3, 768])

In [9]:
c = 1
def residual_stream_patching_hook(
    resid,
    hook,
):
    h_p = pos_cache[hook.name]
    h_n = neg_cache[hook.name]
    steering = h_p - h_n

    len_steering = steering.shape[1]
    
    # resid shape is (batch, pos, d_model)
    # print(resid.shape, steering.shape)
    resid[:, :len_steering, :] =  resid[:, :len_steering, :] + c * steering

    return resid

In [11]:
text = "Yesterday I said"
n_samples = 5
hook_name = utils.get_act_name("resid_post", layer)

with model.hooks(fwd_hooks=[(hook_name, residual_stream_patching_hook)]):
    # model('testing testing asdf;lkjasdf;l k')
    for i in range(n_samples):
        print("============")
        output = model.generate(text, prepend_bos=True, use_past_kv_cache=False, max_new_tokens=20, verbose=False)
        print(output)

Yesterday I said you guys were a 'potentially massive amount of money'. But (I'm perfectly confident you did
Yesterday I said it: "hee-hee! DEA. Hold your shit." Truth be told, when you faced
Yesterday I said...

Refia's last "weight loss" was during the January 11th, 2016 Gulf
Yesterday I said: THE GAMES MADE ME TO CARE FOREVER OF LETHAL. I am completely
Yesterday I said before, there aren't only economics in The Porn Business. Some economists are super passionate about what the


In [13]:
love_words = ["love", "like", "adore", "enjoy", "appreciate", "cherish", "admire", "care", "fancy", "favor", "prefer"]
hate_words = ["hate", "dislike", "detest", "abhor", "despise", "scorn", "loathe", "despise", "fuck you"]

def compute_metric(positive_words, layer, n_samples):
    count = 0
    hook_name = utils.get_act_name("resid_post", layer)

    with model.hooks(fwd_hooks=[(hook_name, residual_stream_patching_hook)]):
        for i in range(n_samples):
            output = model.generate(text, prepend_bos=True, use_past_kv_cache=False, max_new_tokens=30, verbose=False)
            for word in positive_words:
                if word in output.lower():
                    count += 1
                    break
        
    return count/n_samples

for l in range(model.cfg.n_layers):
    n_samples = 2
    score = compute_metric(love_words, l, n_samples)
    print(f'layer: {l}, socre: {score}')

layer: 0, socre: 0.0


In [59]:
steering.shape

torch.Size([1, 3, 768])

12